# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# IMPORTS

In [ ]:
from modules.youtube.extract.ytbpy import channel, playlist, utils, video
from modules import db_manager
from datetime import datetime
from jinja2 import Template
from tqdm import tqdm
import pandas as pd
import json
import yt_dlp
import time
import os

path_read1          = "repository\\querys\\operation\\read\\video.sql"
path_read2          = "repository\\querys\\operation\\read\\video_historic.sql"
path_insert         = "repository\\querys\\operation\\insert\\video_historic.sql"
path_update         = "repository\\querys\\operation\\update\\video_historic.sql"

# lendo tabela de amaração dos videos com as playlist
with open(path_read1, "r", encoding="utf-8") as file:
    query_read1     = file.read()

# lendo tabela de videos finais
with open(path_read2, "r", encoding="utf-8") as file:
    query_read2     = file.read()
    
with open(path_insert, "r", encoding="utf-8") as file:
    query_insert    = file.read()
   
with open(path_update, "r", encoding="utf-8") as file:
    query_update    = file.read()


# PREENCHENDO DADOS DE VIDEOS NOVOS

In [ ]:
df_videos1      = db_manager.read_df(query_read1)
df_videos2      = db_manager.read_df(query_read2)
df_filtrado = df_videos1[~df_videos1['id_movie'].isin(df_videos2['id_movie'])].copy()
for idx, (i, row) in enumerate(tqdm(df_filtrado.iterrows(), total=df_filtrado.shape[0], desc="Processing videos")):
    time.sleep(1)
    video_info = video.get_video_info(url_or_id=row["url"], timeout=10)
    ID_PLAYLIST     = row["id_playlist"] if "id_playlist" in row else None
    TIME_MOVIE      = video_info["duration_seconds"] if "duration_seconds" in video_info else -1
    PUBLISHED_DATE  = video_info["published_date"] if "published_date" in video_info else "" 

    TITLE               = video_info["title"].replace("'", "").strip() if "title" in video_info else ""
    DESCRIPTION         = video_info["description"].replace("'", "").strip() if "description" in video_info else None
    VIEWS               = video_info["views_count"] if "views_count" in video_info else None
    LIKES               = video_info["likes_count"] if "likes_count" in video_info else None
    LIVE                = video_info["is_live"] if "is_live" in video_info else None
    KEYWORDS            = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["keywords"]])+"'" if "keywords" in video_info else None
    AVAILABLE_COUNTRIES = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["available_countries"]])+"'" if "available_countries" in video_info else None
    CATEGORY            = video_info["category"] if "category" in video_info else None
    FAMILY_FRIENDLY     = video_info["family_friendly"] if "family_friendly" in video_info else None
    DT_UPLOAD           = datetime.fromisoformat(video_info["upload_date_text"]).date().isoformat() if "upload_date_text" in video_info else None
    DT_PUBLISH          = datetime.fromisoformat(video_info["publish_date_text"]).date().isoformat() if "publish_date_text" in video_info else None
    DT_START            = datetime.now().strftime("%Y-%m-%d")
    DT_END              = '1900-01-01'
    
    sql_template = Template(query_insert)
    rendered_sql = sql_template.render(
        ID_MOVIE            = row["id_movie"],
        TITLE               = TITLE,
        DESCRIPTION         = DESCRIPTION,
        VIEWS               = VIEWS,
        LIKES               = LIKES,
        LIVE                = LIVE,
        KEYWORDS            = KEYWORDS,
        AVAILABLE_COUNTRIES = AVAILABLE_COUNTRIES,
        CATEGORY            = CATEGORY,
        FAMILY_FRIENDLY     = FAMILY_FRIENDLY,
        DT_UPLOAD           = DT_UPLOAD,
        DT_PUBLISH          = DT_PUBLISH,
        DT_START            = DT_START,
        DT_END              = DT_END
    ).replace("'None'", "NULL").replace("None", "NULL")\
     .replace("'False'", "FALSE").replace("False", "FALSE")\
     .replace("'True'", "TRUE").replace("True", "TRUE")

    try:
        if TITLE != '' and VIEWS is not None and LIKES is not None:
            db_manager.execute_raw(rendered_sql)
        else:
            raise ValueError(f"Dados em branco para o vídeo ID_MOVIE={row['id_movie']}: TITLE='{TITLE}', VIEWS={VIEWS}, LIKES={LIKES}")
    except (Exception, ValueError):
        time.sleep(120)
        db_manager.connection.rollback()


# TENTANDO PREENCHER DADOS EM BRANCO NO BANCO 

In [ ]:
df_videos1      = db_manager.read_df(query_read1)
df_videos2      = db_manager.read_df(query_read2)
videos_null     = df_videos2[df_videos2["views"].isna()].copy()
df_videos2_ok   = df_videos1[df_videos1['id_movie'].isin(videos_null['id_movie'])].copy()

for idx, (i, row) in enumerate(tqdm(df_videos2_ok.iterrows(), total=df_videos2_ok.shape[0], desc="Processing videos")):
    time.sleep(1)
    video_info = video.get_video_info(url_or_id=row["url"], timeout=10)
    ID_PLAYLIST     = row["id_playlist"] if "id_playlist" in row else None
    TIME_MOVIE      = video_info["duration_seconds"] if "duration_seconds" in video_info else -1
    PUBLISHED_DATE  = video_info["published_date"] if "published_date" in video_info else "" 
    
    TITLE               = video_info["title"].replace("'", "").strip() if "title" in video_info else ""
    DESCRIPTION         = video_info["description"].replace("'", "").strip() if "description" in video_info else None
    VIEWS               = video_info["views_count"] if "views_count" in video_info else None
    LIKES               = video_info["likes_count"] if "likes_count" in video_info else None
    LIVE                = video_info["is_live"] if "is_live" in video_info else None
    KEYWORDS            = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["keywords"]])+"'" if "keywords" in video_info else None
    AVAILABLE_COUNTRIES = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["available_countries"]])+"'" if "available_countries" in video_info else None
    CATEGORY            = video_info["category"] if "category" in video_info else None
    FAMILY_FRIENDLY     = video_info["family_friendly"] if "family_friendly" in video_info else None
    DT_UPLOAD           = datetime.fromisoformat(video_info["upload_date_text"]).date().isoformat() if "upload_date_text" in video_info else None
    DT_PUBLISH          = datetime.fromisoformat(video_info["publish_date_text"]).date().isoformat() if "publish_date_text" in video_info else None
    DT_START            = datetime.now().strftime("%Y-%m-%d")
    DT_END              = '1900-01-01'

    sql_template = Template(query_update)
    rendered_sql = sql_template.render(
        ID_MOVIE            = row["id_movie"],
        TITLE               = TITLE,
        DESCRIPTION         = DESCRIPTION,
        VIEWS               = VIEWS,
        LIKES               = LIKES,
        LIVE                = LIVE,
        KEYWORDS            = KEYWORDS,
        AVAILABLE_COUNTRIES = AVAILABLE_COUNTRIES,
        CATEGORY            = CATEGORY,
        FAMILY_FRIENDLY     = FAMILY_FRIENDLY,
        DT_UPLOAD           = DT_UPLOAD,
        DT_PUBLISH          = DT_PUBLISH,
        DT_START            = DT_START,
        DT_END              = DT_END
    ).replace("'None'", "NULL").replace("None", "NULL")\
     .replace("'False'", "FALSE").replace("False", "FALSE")\
     .replace("'True'", "TRUE").replace("True", "TRUE")
    
    try:
        if TITLE != '' and VIEWS is not None and LIKES is not None:
            db_manager.execute_raw(rendered_sql)
        else:
            raise ValueError(f"Dados em branco para o vídeo ID_MOVIE={row['id_movie']}: TITLE='{TITLE}', VIEWS={VIEWS}, LIKES={LIKES}")
    except (Exception, ValueError):
        time.sleep(120)
        db_manager.connection.rollback()


# ATUALIZANDO VALORES DO BANCO

In [ ]:
df_videos1      = db_manager.read_df(query_read1).iloc[15208:]

# Atualizando os videos
for idx, (i, row) in enumerate(tqdm(df_videos1.iterrows(), total=df_videos1.shape[0], desc="Processing videos")):
    time.sleep(1)
    video_info = video.get_video_info(url_or_id=row["url"], timeout=10)
    ID_PLAYLIST     = row["id_playlist"] if "id_playlist" in row else None
    TIME_MOVIE      = video_info["duration_seconds"] if "duration_seconds" in video_info else -1
    PUBLISHED_DATE  = video_info["published_date"] if "published_date" in video_info else "" 
    
    TITLE               = video_info["title"].replace("'", "").strip() if "title" in video_info else ""
    DESCRIPTION         = video_info["description"].replace("'", "").strip() if "description" in video_info else None
    VIEWS               = video_info["views_count"] if "views_count" in video_info else None
    LIKES               = video_info["likes_count"] if "likes_count" in video_info else None
    LIVE                = video_info["is_live"] if "is_live" in video_info else None
    KEYWORDS            = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["keywords"]])+"'" if "keywords" in video_info else None
    AVAILABLE_COUNTRIES = "'"+"', '".join([row.replace("'", "").strip() for row in video_info["available_countries"]])+"'" if "available_countries" in video_info else None
    CATEGORY            = video_info["category"] if "category" in video_info else None
    FAMILY_FRIENDLY     = video_info["family_friendly"] if "family_friendly" in video_info else None
    DT_UPLOAD           = datetime.fromisoformat(video_info["upload_date_text"]).date().isoformat() if "upload_date_text" in video_info else None
    DT_PUBLISH          = datetime.fromisoformat(video_info["publish_date_text"]).date().isoformat() if "publish_date_text" in video_info else None
    DT_START            = datetime.now().strftime("%Y-%m-%d")
    DT_END              = '1900-01-01'

    sql_template = Template(query_update)
    rendered_sql = sql_template.render(
        ID_MOVIE            = row["id_movie"],
        TITLE               = TITLE,
        DESCRIPTION         = DESCRIPTION,
        VIEWS               = VIEWS,
        LIKES               = LIKES,
        LIVE                = LIVE,
        KEYWORDS            = KEYWORDS,
        AVAILABLE_COUNTRIES = AVAILABLE_COUNTRIES,
        CATEGORY            = CATEGORY,
        FAMILY_FRIENDLY     = FAMILY_FRIENDLY,
        DT_UPLOAD           = DT_UPLOAD,
        DT_PUBLISH          = DT_PUBLISH,
        DT_START            = DT_START,
        DT_END              = DT_END
    ).replace("'None'", "NULL").replace("None", "NULL")\
     .replace("'False'", "FALSE").replace("False", "FALSE")\
     .replace("'True'", "TRUE").replace("True", "TRUE")
    
    try:
        if TITLE != '' and VIEWS is not None and LIKES is not None:
            db_manager.execute_raw(rendered_sql)
        else:
            raise ValueError(f"Dados em branco para o vídeo ID_MOVIE={row['id_movie']}: TITLE='{TITLE}', VIEWS={VIEWS}, LIKES={LIKES}")
    except (Exception, ValueError) as e:
        print(f"Error: {e}")
        time.sleep(120)
        db_manager.connection.rollback()


Processing videos:   4%|▍         | 3017/69989 [7:23:13<2252:03:38, 121.06s/it]

# REFRESH VIEWS

In [ ]:
with open("repository\\querys\\operation\\update\\refresh_views.sql","r", encoding="utf-8") as file:
    query_refresh = file.read()
    db_manager.execute_raw(query_refresh)